# Notebook 3 : Analyse pour la Construction de la Couche Gold

## Objectif

Ce notebook vous guide dans l'**analyse des données de la couche silver** pour préparer la construction de la couche **gold** (analytics/BI).

La couche gold contient des données transformées, nettoyées et optimisées pour l'analyse métier et la création de rapports/dashboards.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Exécuté le notebook `2_[LOAD]_load_to_bigquery.ipynb`** pour avoir toutes les tables dans BigQuery (dataset `silver`)
2. **Fichier `.env` configuré** avec les variables d'environnement nécessaires
3. **Service Account** avec les permissions BigQuery (`BigQuery Data Viewer`, `BigQuery Job User`)
4. **Packages Python installés** : `google-cloud-bigquery`, `pandas`, `db-dtypes`, etc.

## Structure du Notebook

Ce notebook contient **4 tâches principales** à réaliser :

1. **Tâche 1 : Analyser la Granularité** - Comprendre le niveau de détail de chaque table
2. **Tâche 2 : Identifier les Transformations** - Déterminer les transformations nécessaires pour la couche gold
3. **Tâche 3 : Identifier les Clés de Jointure** - Mapper les relations entre les tables
4. **Tâche 4 : Autres Analyses** - Qualité des données, agrégations possibles, etc.

## Résultats Attendus

À la fin de ce notebook, vous devriez avoir :
- Une compréhension claire de la structure et de la granularité de chaque table
- Une liste des transformations à appliquer pour créer la couche gold
- Un schéma de jointures documenté
- Des recommandations pour l'optimisation et l'agrégation des données


## Configuration et Connexion à BigQuery

Cette section configure l'environnement et établit la connexion avec BigQuery pour explorer les données de la couche silver.


In [ ]:
# Standard library imports
import os
from pathlib import Path

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

# Configuration
load_dotenv()

ROOT = Path.cwd().parent
PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
DATASET_ID = "silver"

# Authentification
creds = service_account.Credentials.from_service_account_file(SA_PATH)
bq_client = bigquery.Client(project=PROJECT_ID, credentials=creds)

print(f"[OK] - Connecté au projet: {PROJECT_ID}")
print(f"[OK] - Dataset: {DATASET_ID}")


---

## Tâche 1 : Analyser la Granularité de Chaque Table

### Objectif

La **granularité** d'une table correspond au niveau de détail des données qu'elle contient. Comprendre la granularité est essentiel pour :
- Déterminer comment agréger les données
- Identifier les duplications potentielles
- Comprendre le niveau de détail nécessaire pour les analyses métier

### Instructions

Pour chaque table du dataset `silver`, vous devez :

1. **Lister les colonnes** et leurs types
2. **Identifier les clés primaires** ou les colonnes qui identifient de manière unique une ligne
3. **Déterminer la granularité** : à quel niveau de détail sont les données ?
   - Exemple : `fact_validations` pourrait être au niveau **jour × gare × type de titre**
4. **Compter les lignes** et estimer la taille des données
5. **Identifier les colonnes de dimension** (références vers d'autres tables)

### Exemple de Format de Réponse

```
Table: dim_gare
- Granularité: 1 ligne = 1 gare
- Clé primaire: id_gares
- Nombre de lignes: 1234
- Colonnes de dimension: aucune (table de dimension)
```


### À Compléter : Analyse de Granularité

**Tables de Dimension :**

1. `dim_gare`
2. `dim_ligne`
3. `dim_arret`
4. `dim_vacances_scolaires`
5. `dim_transporteur`

**Tables de Fait :**

6. `fact_validations_*` (toutes les tables de validation)

**Votre tâche :** Exécutez des requêtes SQL pour analyser chaque table et remplir le tableau ci-dessous.


In [ ]:
# Exemple : Analyser la table dim_gare
query = f"""
SELECT 
    COUNT(*) as nb_lignes,
    COUNT(DISTINCT id_gares) as nb_gares_uniques,
    COUNT(*) - COUNT(id_gares) as nb_id_null
FROM `{PROJECT_ID}.{DATASET_ID}.dim_gare`
"""

df = bq_client.query(query).to_dataframe()
print("=== Analyse de dim_gare ===")
display(df)

# Afficher le schéma
table = bq_client.get_table(f"{PROJECT_ID}.{DATASET_ID}.dim_gare")
print("\n=== Schéma ===")
for field in table.schema:
    print(f"  - {field.name}: {field.field_type} ({field.mode})")


In [ ]:
# TODO : Répétez cette analyse pour toutes les tables
# Créez un DataFrame pour documenter vos résultats

granularite_analysis = []

# Exemple pour dim_gare
granularite_analysis.append({
    "table": "dim_gare",
    "type": "dimension",
    "granularite": "1 ligne = 1 gare",
    "cle_primaire": "id_gares",
    "nb_lignes": 0,  # À compléter
    "colonnes_dimension": "aucune"
})

# TODO : Ajoutez les autres tables
# - dim_ligne
# - dim_arret
# - dim_vacances_scolaires
# - dim_transporteur
# - fact_validations_* (analyser au moins une table de validation)

df_granularite = pd.DataFrame(granularite_analysis)
display(df_granularite)


---

## Tâche 2 : Identifier les Transformations Nécessaires

### Objectif

Identifier les **transformations** à appliquer aux données de la couche silver pour créer la couche gold optimisée pour l'analyse.

### Types de Transformations Possibles

1. **Nettoyage des données**
   - Suppression des doublons
   - Gestion des valeurs NULL
   - Normalisation des formats (dates, textes)

2. **Enrichissement**
   - Ajout de colonnes calculées
   - Jointures avec les tables de dimension
   - Ajout de catégories/segments

3. **Agrégation**
   - Regroupement par dimensions (jour, gare, ligne, etc.)
   - Calcul de métriques (somme, moyenne, comptage)
   - Création de tables pré-agrégées

4. **Normalisation**
   - Uniformisation des formats de dates
   - Standardisation des noms de colonnes
   - Conversion de types de données

5. **Déduplication**
   - Identification et suppression des doublons
   - Consolidation de données dupliquées

### Instructions

Pour chaque table, identifiez :
1. **Les problèmes de qualité** à corriger
2. **Les transformations nécessaires** avec des exemples concrets
3. **Les colonnes à ajouter** (calculées ou issues de jointures)
4. **Les agrégations possibles** pour optimiser les requêtes


In [ ]:
# Exemple 1 : Vérifier les doublons dans fact_validations
query = f"""
SELECT 
    JOUR,
    ID_ZDC,
    CATEGORIE_TITRE,
    COUNT(*) as nb_occurrences
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt`
GROUP BY JOUR, ID_ZDC, CATEGORIE_TITRE
HAVING COUNT(*) > 1
LIMIT 10
"""

df_duplicates = bq_client.query(query).to_dataframe()
print("=== Exemple : Recherche de doublons ===")
display(df_duplicates)


In [ ]:
# Exemple 2 : Vérifier les valeurs NULL
query = f"""
SELECT 
    COUNT(*) as total_lignes,
    COUNT(JOUR) as jour_not_null,
    COUNT(ID_ZDC) as id_zdc_not_null,
    COUNT(NB_VALD) as nb_vald_not_null,
    COUNT(*) - COUNT(JOUR) as jour_null,
    COUNT(*) - COUNT(ID_ZDC) as id_zdc_null,
    COUNT(*) - COUNT(NB_VALD) as nb_vald_null
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt`
"""

df_nulls = bq_client.query(query).to_dataframe()
print("=== Exemple : Analyse des valeurs NULL ===")
display(df_nulls)


In [ ]:
# TODO : Documentez les transformations nécessaires

transformations = []

# Exemple pour fact_validations
transformations.append({
    "table_source": "fact_validations_*",
    "probleme": "Format de date en STRING (DD/MM/YYYY)",
    "transformation": "Convertir JOUR en DATE avec PARSE_DATE('%d/%m/%Y', JOUR)",
    "exemple_sql": "SELECT PARSE_DATE('%d/%m/%Y', JOUR) as date_validation FROM ...",
    "priorite": "Haute"
})

transformations.append({
    "table_source": "fact_validations_*",
    "probleme": "Colonnes ID_ZDC, lda, ID_REFA_LDA non normalisées",
    "transformation": "Unifier les noms de colonnes (toutes → id_zdc)",
    "exemple_sql": "SELECT COALESCE(ID_ZDC, lda, ID_REFA_LDA) as id_zdc FROM ...",
    "priorite": "Haute"
})

# TODO : Ajoutez d'autres transformations identifiées
# - Conversion de types (STRING → INTEGER pour NB_VALD)
# - Normalisation des noms de colonnes
# - Ajout de colonnes calculées (jour_semaine, est_vacances, etc.)
# - Unification des tables fact_validations_*

df_transformations = pd.DataFrame(transformations)
display(df_transformations)


---

## Tâche 3 : Identifier les Clés de Jointure Possibles

### Objectif

Identifier toutes les **relations possibles** entre les tables pour pouvoir créer des jointures dans la couche gold.

### Instructions

Pour chaque paire de tables, identifiez :

1. **Les colonnes de jointure** (clés étrangères)
2. **Le type de relation** (1-1, 1-N, N-N)
3. **La cardinalité** (combien de lignes de la table A correspondent à combien de lignes de la table B)
4. **Vérifier l'intégrité référentielle** (toutes les clés étrangères existent-elles dans la table de dimension ?)

### Exemples de Relations à Identifier

- `fact_validations` ↔ `dim_gare` : via `ID_ZDC` ou `id_gares`
- `fact_validations` ↔ `dim_ligne` : via `CODE_STIF_TRNS` ou `idrefliga`
- `fact_validations` ↔ `dim_arret` : via `CODE_STIF_ARRET`
- `dim_gare` ↔ `dim_ligne` : via `idrefliga`
- `dim_ligne` ↔ `dim_transporteur` : via un code transporteur
- `fact_validations` ↔ `dim_vacances_scolaires` : via la date
- `fact_validations` ↔ `dim_jours_feries` : via la date (si la table existe)


In [ ]:
# Exemple 1 : Vérifier la jointure fact_validations ↔ dim_gare
query = f"""
SELECT 
    COUNT(DISTINCT v.ID_ZDC) as nb_id_zdc_distincts_fact,
    COUNT(DISTINCT g.id_gares) as nb_id_gares_distincts_dim,
    COUNT(DISTINCT v.ID_ZDC) - COUNT(DISTINCT g.id_gares) as difference
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt` v
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.dim_gare` g
    ON CAST(v.ID_ZDC AS INT64) = g.id_gares
"""

df_join_check = bq_client.query(query).to_dataframe()
print("=== Exemple : Vérification de la jointure fact_validations ↔ dim_gare ===")
display(df_join_check)


In [ ]:
# Exemple 2 : Identifier les clés orphelines (clés étrangères sans correspondance)
query = f"""
SELECT 
    v.ID_ZDC,
    COUNT(*) as nb_occurrences
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt` v
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.dim_gare` g
    ON CAST(v.ID_ZDC AS INT64) = g.id_gares
WHERE g.id_gares IS NULL
GROUP BY v.ID_ZDC
ORDER BY nb_occurrences DESC
LIMIT 10
"""

df_orphans = bq_client.query(query).to_dataframe()
print("=== Exemple : Clés orphelines (sans correspondance dans dim_gare) ===")
display(df_orphans)


In [ ]:
# TODO : Documentez toutes les relations identifiées

jointures = []

# Exemple 1
jointures.append({
    "table_source": "fact_validations",
    "table_cible": "dim_gare",
    "colonne_source": "ID_ZDC",
    "colonne_cible": "id_gares",
    "type_relation": "N-1",
    "description": "Plusieurs validations peuvent être associées à une même gare",
    "integrite": "À vérifier"
})

# Exemple 2
jointures.append({
    "table_source": "fact_validations",
    "table_cible": "dim_ligne",
    "colonne_source": "CODE_STIF_TRNS",
    "colonne_cible": "À identifier",
    "type_relation": "N-1",
    "description": "Plusieurs validations peuvent être associées à une même ligne",
    "integrite": "À vérifier"
})

# TODO : Ajoutez toutes les autres relations identifiées
# - fact_validations ↔ dim_arret
# - fact_validations ↔ dim_vacances_scolaires (via date)
# - fact_validations ↔ dim_jours_feries (via date, si existe)
# - dim_gare ↔ dim_ligne
# - dim_ligne ↔ dim_transporteur
# - etc.

df_jointures = pd.DataFrame(jointures)
display(df_jointures)


---

## Tâche 4 : Autres Analyses et Recommandations

### Objectif

Effectuer des analyses complémentaires pour optimiser la construction de la couche gold.

### Tâches Suggérées

#### 4.1 - Analyse de la Qualité des Données

- **Complétude** : Pourcentage de valeurs non NULL par colonne
- **Cohérence** : Vérifier les valeurs aberrantes (ex: dates futures, nombres négatifs)
- **Unicité** : Identifier les colonnes qui devraient être uniques mais ne le sont pas
- **Validité** : Vérifier que les valeurs respectent les contraintes métier

#### 4.2 - Analyse des Agrégations Possibles

Identifier les **niveaux d'agrégation** utiles pour les analyses :
- Par jour, semaine, mois, année
- Par gare, ligne, arrêt, transporteur
- Par type de titre (catégorie)
- Combinaisons de ces dimensions

#### 4.3 - Analyse de Performance

- **Volume de données** : Taille des tables, nombre de lignes
- **Fréquence d'utilisation** : Quelles tables seront les plus sollicitées ?
- **Optimisations possibles** : Partitionnement, clustering, tables matérialisées

#### 4.4 - Identification des Métriques Métier

Identifier les **KPIs** (Key Performance Indicators) à calculer :
- Nombre total de validations par période
- Taux de croissance des validations
- Répartition par type de titre
- Top 10 des gares les plus fréquentées
- Comparaison jour ouvrable vs jour férié vs vacances

#### 4.5 - Proposition de Schéma Gold

Proposer une **structure de tables** pour la couche gold :
- Tables de fait agrégées (ex: `fact_validations_daily`)
- Tables de dimension enrichies (ex: `dim_gare_enriched`)
- Tables de métriques pré-calculées (ex: `metrics_affluence_weekly`)
- Vues pour faciliter les requêtes courantes


In [ ]:
# Exemple : Analyse de complétude des données
query = f"""
SELECT 
    'fact_validations_2023_s2_nb_fer_txt' as table_name,
    COUNT(*) as total_lignes,
    ROUND(100.0 * COUNT(JOUR) / COUNT(*), 2) as pct_jour_not_null,
    ROUND(100.0 * COUNT(ID_ZDC) / COUNT(*), 2) as pct_id_zdc_not_null,
    ROUND(100.0 * COUNT(NB_VALD) / COUNT(*), 2) as pct_nb_vald_not_null,
    ROUND(100.0 * COUNT(CATEGORIE_TITRE) / COUNT(*), 2) as pct_categorie_not_null
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt`
"""

df_quality = bq_client.query(query).to_dataframe()
print("=== Exemple : Analyse de complétude ===")
display(df_quality)


In [ ]:
# Exemple : Identifier les niveaux d'agrégation possibles
query = f"""
SELECT 
    COUNT(DISTINCT JOUR) as nb_jours_distincts,
    COUNT(DISTINCT ID_ZDC) as nb_gares_distincts,
    COUNT(DISTINCT CODE_STIF_TRNS) as nb_lignes_distincts,
    COUNT(DISTINCT CATEGORIE_TITRE) as nb_categories_distincts,
    COUNT(*) as total_validations
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt`
"""

df_aggregations = bq_client.query(query).to_dataframe()
print("=== Exemple : Dimensions disponibles pour agrégation ===")
display(df_aggregations)

print("\n=== Niveaux d'agrégation possibles ===")
print("- Par jour : ~", df_aggregations['nb_jours_distincts'].iloc[0], "combinaisons")
print("- Par jour × gare : ~", df_aggregations['nb_jours_distincts'].iloc[0] * df_aggregations['nb_gares_distincts'].iloc[0], "combinaisons")
print("- Par jour × gare × catégorie : ~", df_aggregations['nb_jours_distincts'].iloc[0] * df_aggregations['nb_gares_distincts'].iloc[0] * df_aggregations['nb_categories_distincts'].iloc[0], "combinaisons")


In [ ]:
# Exemple : Calculer des métriques métier
query = f"""
SELECT 
    PARSE_DATE('%d/%m/%Y', JOUR) as date_validation,
    COUNT(DISTINCT ID_ZDC) as nb_gares,
    SUM(CAST(NB_VALD AS INT64)) as total_validations,
    AVG(CAST(NB_VALD AS FLOAT64)) as moyenne_validations_par_ligne
FROM `{PROJECT_ID}.{DATASET_ID}.fact_validations_2023_s2_nb_fer_txt`
WHERE JOUR IS NOT NULL AND NB_VALD IS NOT NULL
GROUP BY date_validation
ORDER BY date_validation
LIMIT 10
"""

df_metrics = bq_client.query(query).to_dataframe()
print("=== Exemple : Métriques métier (validations par jour) ===")
display(df_metrics)


### À Compléter : Recommandations pour la Couche Gold

**Votre tâche :** Remplissez ce tableau avec vos recommandations basées sur vos analyses.


In [ ]:
# TODO : Documentez vos recommandations

recommandations = []

# Exemple 1
recommandations.append({
    "categorie": "Transformation",
    "recommandation": "Unifier toutes les tables fact_validations_* en une seule table",
    "justification": "Faciliter les requêtes et analyses cross-années",
    "priorite": "Haute"
})

# Exemple 2
recommandations.append({
    "categorie": "Agrégation",
    "recommandation": "Créer une table fact_validations_daily agrégée par jour × gare × catégorie",
    "justification": "Réduire le volume de données et accélérer les requêtes d'analyse",
    "priorite": "Moyenne"
})

# Exemple 3
recommandations.append({
    "categorie": "Enrichissement",
    "recommandation": "Ajouter des colonnes calculées : jour_semaine, est_vacances, est_ferie",
    "justification": "Faciliter les analyses comparatives (weekend vs semaine, vacances vs période scolaire)",
    "priorite": "Haute"
})

# TODO : Ajoutez vos propres recommandations
# - Optimisations de performance (partitionnement, clustering)
# - Nettoyage de données
# - Création de vues
# - Tables de métriques pré-calculées
# - etc.

df_recommandations = pd.DataFrame(recommandations)
display(df_recommandations)


---

## Conclusion et Prochaines Étapes

### Résumé de Votre Travail

À la fin de ce notebook, vous devriez avoir produit :

1. ✅ **Analyse de granularité** : Documentation complète de chaque table
2. ✅ **Liste des transformations** : Transformations nécessaires avec exemples SQL
3. ✅ **Schéma de jointures** : Toutes les relations identifiées entre les tables
4. ✅ **Recommandations** : Propositions pour optimiser la couche gold

### Prochaines Étapes

Une fois ces analyses terminées, vous pourrez :

1. **Créer le notebook de transformation** (`4_[TRANSFORM]_build_gold.ipynb`)
2. **Implémenter les transformations identifiées**
3. **Créer les tables de la couche gold** dans BigQuery
4. **Valider la qualité des données** transformées
5. **Créer des vues** pour faciliter les requêtes d'analyse

### Ressources Utiles

- [Documentation BigQuery SQL](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax)
- [Fonctions de date BigQuery](https://cloud.google.com/bigquery/docs/reference/standard-sql/date_functions)
- [Fonctions d'agrégation BigQuery](https://cloud.google.com/bigquery/docs/reference/standard-sql/aggregate_functions)
- [Partitionnement et clustering BigQuery](https://cloud.google.com/bigquery/docs/partitioned-tables)

---

**Bon travail ! 🚀**
